In [59]:
# Author: Robert Guthrie

import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim

torch.manual_seed(1)

In [60]:
def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()


def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)


# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

In [156]:
class BiLSTM_CRF(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim):
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)

        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=1, bidirectional=True, batch_first=True)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.init_hidden()

    def init_hidden(self):

        #TODO
        return (torch.randn(2, 2, self.hidden_dim // 2),
                torch.randn(2, 2, self.hidden_dim // 2))

    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha

    def _get_lstm_features(self, sentences):
        self.hidden = self.init_hidden()
        embeds = self.word_embeds(sentences)
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats

    # def _score_sentence(self, feats, tags):
    #     # Initialize score tensor of shape [batch_size]
    #     batch_size = feats.shape[0]
    #     seq_len = feats.shape[1]

    #     # Initialize score as a tensor of zeros for each sentence in the batch
    #     score = torch.zeros(batch_size, device=feats.device)

    #     # Prepend the START_TAG to each sequence in the batch
    #     start_tensor = torch.full((batch_size, 1), self.tag_to_ix[START_TAG], dtype=torch.long, device=feats.device)
    #     tags = torch.cat([start_tensor, tags], dim=1)

    #     # Iterate over each timestep in the sequence
    #     for i in range(seq_len):
    #         # Retrieve transition and emission scores, ensuring they are the same shape
    #         transition_score = self.transitions[tags[:, i + 1], tags[:, i]]  # shape: [batch_size]
    #         emission_score = feats[torch.arange(batch_size), i, tags[:, i + 1]]  # shape: [batch_size]

    #         # Update the score by adding the transition and emission scores
    #         score += transition_score + emission_score

    #     # Add the transition to the STOP_TAG
    #     score += self.transitions[self.tag_to_ix[STOP_TAG], tags[:, -1]]

    #     return score

    def _score_sentence(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long), tags])
        for i, feat in enumerate(feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score


    def _viterbi_decode(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_log_likelihood(self, sentence, tags):
        feats = self._get_lstm_features(sentence)
        forward_score = [self._forward_alg(feat) for feat in feats]
        gold_score = [self._score_sentence(feat, tag) for feat, tag in zip(feats, tags)]
        return sum(forward_score)/len(forward_score) - sum(gold_score)/len(gold_score)

    def forward(self, sentence):  # dont confuse this with _forward_alg above.
        # Get the emission scores from the BiLSTM
        lstm_feats = self._get_lstm_features(sentence)

        # Find the best path, given the features.
        score, tag_seq = self._viterbi_decode(lstm_feats)
        return score, tag_seq

In [62]:
import json

with open('../data/Corona2.json') as f:
    d = json.load(f)

dataset_sample = []

for example in d['examples']:
    
    entities = [ (annot['start'], annot['end'], annot['value'], annot['tag_name']) for annot in example['annotations']]
    
    dataset_sample.append({
        'text': example['content'],
        'entities': entities
    })

tags = set()

for example in d['examples']:
    for annot in example['annotations']:
        tags.add(annot['tag_name'])
        
tags = list(tags)
print(tags)

['Medicine', 'Pathogen', 'MedicalCondition']


In [63]:
import torch
import torch.nn as nn
from torch import cuda
from torch.utils.data import DataLoader
from transformers import AutoTokenizer
from preprocess.setup import Preprocess
from TorchCRF import CRF
from torch_struct import LinearChainCRF


In [64]:
checkpoint = "distilbert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

c:\Users\driti\Documents\NTNU\Master\master-project\.venv\lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [65]:
TRAIN_BATCH_SIZE = 2
VALID_BATCH_SIZE = 2
EPOCHS = 3
LEARNING_RATE = 1e-05
MAX_GRAD_NORM = 10

In [66]:
processed = Preprocess(tokenizer).run(dataset_sample, tags)
            
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(processed["train"], **train_params)
testing_loader = DataLoader(processed["test"], **test_params)

In [67]:
tag_to_ix = processed['label2id']

In [157]:
START_TAG = "<START>"
STOP_TAG = "<STOP>"
EMBEDDING_DIM = 5
HIDDEN_DIM = 4

START_ID = max(processed['id2label'].keys()) + 1
STOP_ID = max(processed['id2label'].keys())+2


processed['label2id'][START_TAG] = START_ID
processed['label2id'][STOP_TAG] = STOP_ID

model = BiLSTM_CRF(tokenizer.vocab_size, tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM)
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-4)

device = "cpu"

# Make sure prepare_sequence from earlier in the LSTM section is loaded
for epoch in range(2):  # again, normally you would NOT do 300 epochs, it is toy data
    for idx, batch in enumerate(training_loader):
        
        ids = batch['ids'].to(device, dtype = torch.long)
        mask = batch['mask'].to(device, dtype = torch.long)
        targets = batch['targets'].to(device, dtype = torch.long)
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 3. Run our forward pass.
        loss = model.neg_log_likelihood(ids, targets)

        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()
# We got it!

tensor([1646.9774], grad_fn=<SubBackward0>)
tensor([1312.9026], grad_fn=<SubBackward0>)
tensor([399.2588], grad_fn=<SubBackward0>)
tensor([2844.0710], grad_fn=<SubBackward0>)
tensor([1035.4814], grad_fn=<SubBackward0>)
tensor([2157.1045], grad_fn=<SubBackward0>)
tensor([321.0598], grad_fn=<SubBackward0>)
tensor([809.1028], grad_fn=<SubBackward0>)
tensor([2188.6475], grad_fn=<SubBackward0>)
tensor([806.1801], grad_fn=<SubBackward0>)
tensor([412.6582], grad_fn=<SubBackward0>)
tensor([4375.8457], grad_fn=<SubBackward0>)
tensor([401.2639], grad_fn=<SubBackward0>)
tensor([2686.0945], grad_fn=<SubBackward0>)
tensor([227.7740], grad_fn=<SubBackward0>)
tensor([274.8374], grad_fn=<SubBackward0>)
tensor([1640.7555], grad_fn=<SubBackward0>)
tensor([783.9675], grad_fn=<SubBackward0>)
tensor([4936.1572], grad_fn=<SubBackward0>)
tensor([182.6174], grad_fn=<SubBackward0>)
tensor([231.4536], grad_fn=<SubBackward0>)
tensor([123.8877], grad_fn=<SubBackward0>)
tensor([3093.1196], grad_fn=<SubBackward0>)
